# Lab 10

_[General notebook information](https://computing-in-context.afeld.me/notebooks.html)_

## Objectives

Practice:

- Data cleaning
- [Merging datasets through pandas](https://pandas.pydata.org/docs/user_guide/merging.html#merge)


## Data Cleaning

1. Make messy data.
   1. Open [this shared 311 data](https://docs.google.com/spreadsheets/d/1ZjDYeGqcoXz7AdHQ6N0q4zxMtEXp8JfLvjNFKN6kGtk/edit?gid=1427786321#gid=1427786321).
   1. Make a couple edits to mess it up.
1. Download the data.
   1. Click `File`.
   1. Click `Download`.
   1. Click `Comma Separated Values (.csv)`.

Now this data is chaotic with some of your suggestions. How do we clean such a dataset?


### Guide

- Inspect the Data
- Check missing values
- Fix formatting issues and spaces
- Standardize categoricals
- Handle special characters
- Handle bad [illogical] data


## Merging data

- This is totally separate from the Dat Cleaning above.
- The instructions here are intentionally incomplete.


### Step 0

Find an NYC dataset with a borough column.

- Use [Scout](https://scoutopendata.com/explore/NYC) to filter by column name.
- Don't spend too long on this step.
- [Keep the dataset small](https://python-public-policy.afeld.me/en/columbia/assignments/open_ended.html#reducing-data-size) (under 500,000-ish rows) to make it easier to work with.


**What's the URL of your dataset?**


Film Permits: Permits are generally required when asserting the exclusive use of city property, like a sidewalk, a street, or a park

https://data.cityofnewyork.us/City-Government/Film-Permits/tg4x-b46p/about_data


### Step 1

Save and load the dataset.


In [6]:
import pandas as pd

film = pd.read_csv("Film_Permits_20251111.csv")

# quick checks
film.head()
film.info()
film["Borough"].value_counts(dropna=False)

valid_boros = ["Manhattan", "Brooklyn", "Queens", "Bronx", "Staten Island"]

# convert to string, format so that all boroughs are title cased 
film["Borough"] = (
    film["Borough"]
    .astype(str)
    .str.strip()
    .str.title()
)

# ensuring the boroughs are within the boroughs we've provided
film = film[film["Borough"].isin(valid_boros)]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14207 entries, 0 to 14206
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   EventID            14207 non-null  int64 
 1   EventType          14207 non-null  object
 2   StartDateTime      14207 non-null  object
 3   EndDateTime        14207 non-null  object
 4   EnteredOn          14207 non-null  object
 5   EventAgency        14207 non-null  object
 6   ParkingHeld        14207 non-null  object
 7   Borough            14207 non-null  object
 8   CommunityBoard(s)  14198 non-null  object
 9   PolicePrecinct(s)  14198 non-null  object
 10  Category           14207 non-null  object
 11  SubCategoryName    14207 non-null  object
 12  Country            14207 non-null  object
 13  ZipCode(s)         14198 non-null  object
dtypes: int64(1), object(13)
memory usage: 1.5+ MB


### Step 2

Download and load the [Population by Borough](https://data.cityofnewyork.us/City-Government/New-York-City-Population-by-Borough-1950-2040/xywu-7bv9) dataset.


In [20]:
#reads population file
pop_raw = pd.read_csv("New_York_City_Population_by_Borough,_1950_-_2040_20251111.csv")

#keeping only the 2020 and Borough columns
pop = pop_raw[["Borough", "2020"]].rename(columns={"2020": "Population"})

#cleaning up text to match with burough formatting above
pop["Borough"] = pop["Borough"].str.strip().str.title()

#keeping only valid boroughs like in step 1
pop = pop[pop["Borough"].isin(valid_boros)]

print(pop)

         Borough Population
1          Bronx  1,446,788
2       Brooklyn  2,648,452
3      Manhattan  1,638,281
4         Queens  2,330,295
5  Staten Island    487,155


### Step 3

Use [`merge()`](https://pandas.pydata.org/docs/user_guide/merging.html#merge) to combine the two, and output the resulting table.


In [8]:
#left join on borough keeps all rows from left-hand df (film)
merged = film.merge(pop, on="Borough", how="left")

#display 5 first values
merged.head()

,EventID,EventType,StartDateTime,EndDateTime,EnteredOn,EventAgency,ParkingHeld,Borough,CommunityBoard(s),PolicePrecinct(s),Category,SubCategoryName,Country,ZipCode(s),Population
0,881068,Shooting Permit,08/10/2025 06:00:00 AM,08/10/2025 11:59:00 PM,08/08/2025 01:34:23 PM,Mayor's Office of Media & Entertainment,WEST BROADWAY between BROOME STREET and SPRING...,Manhattan,2,"1, 5",Television,Cable-episodic,United States of America,"10012, 10013","1,638,281"
1,880935,Shooting Permit,08/11/2025 07:00:00 AM,08/11/2025 07:00:00 PM,08/07/2025 04:38:34 PM,Mayor's Office of Media & Entertainment,WEST 26 STREET between 7 AVENUE and 8 AVENUE,Manhattan,"4, 5",10,Television,Reality,United States of America,10001,"1,638,281"
2,880933,Shooting Permit,08/10/2025 06:00:00 AM,08/10/2025 09:00:00 PM,08/07/2025 04:27:52 PM,Mayor's Office of Media & Entertainment,GREENWICH STREET between NORTH MOORE STREET an...,Manhattan,1,1,Still Photography,Not Applicable,United States of America,10013,"1,638,281"
3,880923,Shooting Permit,08/11/2025 07:00:00 AM,08/11/2025 09:00:00 PM,08/07/2025 03:55:19 PM,Mayor's Office of Media & Entertainment,KINGSLAND AVENUE between GREENPOINT AVENUE and...,Brooklyn,1,94,Television,Cable-episodic,United States of America,11222,"2,648,452"
4,880922,Shooting Permit,08/11/2025 12:00:00 PM,08/11/2025 11:59:00 PM,08/07/2025 03:53:15 PM,Mayor's Office of Media & Entertainment,SPRING STREET between GREENWICH STREET and REN...,Manhattan,2,1,Film,Short,United States of America,10013,"1,638,281"


### Step 4

Using the two datasets above, use pandas to produce an aggregate per-capita statistic by borough.

The dataset you chose before may not work for this. That's fine, pick another.

#### Hint

You're creating a "number of [thing] per capita by borough" table.

1. Do a [`groupby()`](https://pandas.pydata.org/docs/user_guide/groupby.html) on the original dataset.
1. Join with the populations by borough.
1. Compute the per-capita values as a new column.


In [69]:
#grping film permit rows by borough, .size for no of permits by borough
permits_by_boro = (
    film
    .groupby("Borough")
    .size()
    .reset_index(name="permit_count")
)

#merging no of permits df created above with borough population data
boro_stats = permits_by_boro.merge(pop, on="Borough", how="left")

#removing commas from numbers
boro_stats["Population"] = (
    boro_stats["Population"]
    .str.replace(",", "")
)

#turing strings into numeric 
boro_stats["Population"] = pd.to_numeric(boro_stats["Population"])

#find average here
boro_stats["permits_per_capita"] = (
    boro_stats["permit_count"] / boro_stats["Population"]
)

#display stats
print(boro_stats)



         Borough  permit_count  Population  permits_per_capita
0          Bronx           406     1446788            0.000281
1       Brooklyn          4616     2648452            0.001743
2      Manhattan          7162     1638281            0.004372
3         Queens          1895     2330295            0.000813
4  Staten Island           128      487155            0.000263


### Step 5

[Submit.](https://computing-in-context.afeld.me/notebooks.html#submission)
